<a href="https://colab.research.google.com/github/rafid29mehda/User-Review-Analyzer/blob/main/Final_Review_Fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Load the uploaded file into a DataFrame
df = pd.read_csv(next(iter(uploaded)))

# Map labels to integers (Functional: 1, Non-Functional: 0)
label_mapping = {'F': 1, 'NF': 0}
df['labels'] = df['RequirementType'].map(label_mapping)

# Split the dataset into train and test
train_df, test_df = train_test_split(df[['content', 'labels']], test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


Saving final_corrected_fine_labeled_reviews.csv to final_corrected_fine_labeled_reviews.csv


In [ ]:
from transformers import DistilBertTokenizer

# Load pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['content'], padding="max_length", truncation=True)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/9996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Ensure using the correct eval strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust this if running out of memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=len(train_dataset) // 8,  # Ensure all steps are logged
)

# Initialize the Trainer again
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model again
trainer.train()


NameError: name 'TrainingArguments' is not defined

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Step 1: Get predictions from the model on the test set
predictions = trainer.predict(test_dataset)

# Step 2: Convert logits to predicted class
preds = np.argmax(predictions.predictions, axis=-1)

# Step 3: Calculate accuracy
accuracy = accuracy_score(test_df['labels'], preds)

# Step 4: Print accuracy
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 99.84%


In [ ]:
trainer.save_model("final_review_model")


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="final_review_model",  # Path to the saved model folder
    repo_id="RafidMehda/final_review_model",  # Your Hugging Face username and model name
    repo_type="model"
)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RafidMehda/final_review_model/commit/8704c36371c2fe98c70c462d2ddd7374507c2a7b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='8704c36371c2fe98c70c462d2ddd7374507c2a7b', pr_url=None, pr_revision=None, pr_num=None)